# Lecture 5: Data Wrangling with Census Data

Often, real-world data isn't in quite the correct format! Sometimes data is in the wrong units, sometimes there are missing values, and sometimes very high/very low values a

Addressing these challenges in order to answer research questions is often referred to as **data wrangling**. Let's walk through an example of how to wrangle some data!

Let's use the Census Public Use Microdata File once again to ask a question:

- **What percentage of their income do households spend on housing each month?**

This question is super important - if households have to spend too much of their money on housing, they won't have any money left over for other things like food, clothes, savings, etc!

Statistics Canada calculates this "shelter-cost-to-income ratio" for each household using by the following formula:

$${\text{Shelter-Cost-to-Income Ratio}} = 100\times \frac{\text{Monthly Shelter Cost}}{\text{Monthly Household Income}} $$

## Step 1: Load Census PUMF Data

Let's begin by importing pandas and reading in our census data:

In [ ]:
import pandas as pd

pumf_data = pd.read_csv("pumf_housing_income.csv")

pumf_data.head()

## Step 2: Subset data

For this example, we will explore the following columns:

   + `HH_ID`: unique household identifier 
   + `CMA`: metro area
   + `TOTINC`: total individual annual income
   + `SHELCO`: monthly household shelter cost 

Just like last week, let's create a subset of `pumf_data` with only these columns!

In [ ]:
important_columns = [
    "HH_ID", # Household ID
    "CMA", # Metro Area
    "TOTINC", # Total income
    "SHELCO" # Shelter cost
]

important_columns = ["HH_ID", "CMA", "TOTINC", "SHELCO"] 

subset_pumf_data = pumf_data[important_columns]

subset_pumf_data.head()

## Step 3: Rename columns

Let's use the `rename` function to rename columns to something more intuitive:

In [ ]:
new_column_names = {
    "HH_ID": "Household ID",
    "CMA": "Metro Area",
    "TOTINC": "Annual Income",
    "SHELCO": "Monthly Housing Costs"
}
                    
subset_pumf_data = subset_pumf_data.rename(columns = new_column_names)

subset_pumf_data.head()

## Step 4: Investigate the data and address any issues

We can use `.describe()` to look at the specific columns we're interested in. 

In this case, let's look specifically at `Annual Income` and `Monthly Housing Costs`:

In [ ]:
subset_pumf_data[["Annual Income", "Monthly Housing Costs"]].describe()

`pandas` automatically displays very large numbers and very small numbers in **scientific notation** (for example, 1+e05 instead of 100,000). This is difficult to interpret! 

We can change the default options within `pandas` so that it display floats (decimal numbers) as full numbers with two decimal places for the rest of this session:

In [ ]:
pd.set_option('display.float_format', '{:.2f}'.format)

Let's try this again:

In [ ]:
subset_pumf_data[["Annual Income", "Monthly Housing Costs"]].describe()

`.describe()` summarizes each of our columns with numeric data. In this case, we are particularly interested in `Annual Income` and `Monthly Housing Costs`. Pay particular attention to the `min` and `max` values for each variable!

- `Monthly Housing Costs` looks good: the minimum is \\$0 and the maximum is \\$4,817, which are both plausible for monthly housing costs.

- For `Annual Income`, the minimum value is -\\$50,000, and the maximum value is \\$99,999,999. This doesn't seem very believable!

- According to the <a href="2021 Census Hierarchical PUMF User Guide_V2.pdf" target="_blank">codebook</a>, the value 99,999,999 is a **placeholder** value for people who are younger than 15 years old, and aren't expected to have any income!

- There is also another placeholder value (88,888,888) for values that are simply *missing*, likely because someone didn't answer that question on the survey.

    - We should **remove** any cases with these numbers, because those aren't actual incomes! We can do this by looking for values that are **not equal to 88888888** and **not equal to 99999999**:

In [ ]:
# Create a boolean for rows with an actual income
actual_income = (subset_pumf_data["Annual Income"] != 88888888) & (subset_pumf_data["Annual Income"] != 99999999)

# Count original number of rows
print(f"The total number of rows is: {len(subset_pumf_data)}")

# Count number of values where annual income is not a placeholder
print(f"The total number of rows with an actual income is: {actual_income.sum()}")

Now, let's filter our dataset to only keep rows with an actual income value!

In [ ]:
# Filter subset_pumf_data using our `actual_income` boolean
subset_pumf_data = subset_pumf_data[actual_income]

# Count number of rows to confirm this worked
print(len(subset_pumf_data))

Now, let's use `.describe()` to check the values in our columns again:

In [ ]:
subset_pumf_data[["Annual Income", "Monthly Housing Costs"]].describe()

Now, our maximum income value is \\$1,039,418. This is much more believable!

There are also negative incomes, but we'll deal with those later.

## Step 5: Use .groupby to summarize household incomes

Right now, we have records for **individual people** - notice how there are multiple records for Households 2 and 3: 

In [ ]:
subset_pumf_data.head()

The thing is, we need to know the total amount of income that each *household* is making in total!

Let's use `.groupby()` to group our data frame **by household**. This creates a `DataFrameGroupBy` object:

In [ ]:
grouped_by_hh = subset_pumf_data.groupby("Household ID")

type(grouped_by_hh)

Then, we can add up the total income for each household, by taking the `Annual Income` column from this grouped object and calculating the `.sum()`:

In [ ]:
grouped_by_hh["Annual Income"].sum()

- By default, results from a groupby object will print as a pandas Series. If we want a pandas data frame instead, we should include the argument `as_index = False` inside of the `groupby()` function.

- Let's also include `Metro Area` and `Monthly Housing Costs` inside our `.groupby()` command, because those values will always be the same for each household member: 
    - Each household member lives within the same Metro Area, and each household member has the same overall housing cost.

In [ ]:
grouped_by_hh = subset_pumf_data.groupby(["Household ID", "Metro Area", "Monthly Housing Costs"], as_index = False)

pumf_summary = grouped_by_hh["Annual Income"].sum()

pumf_summary

## Step 6: Remove zero/negative incomes

As a reminder, we are trying to calculate the following formula:

$${\text{Shelter-Cost-to-Income Ratio}} = 100\times \frac{\text{Monthly Shelter Cost}}{\text{Monthly Household Income}} $$

- When calculating the percentage of income spent on rent, Statistics Canada advises removing any households that have incomes that are negative or \\$0. 

- It is impossible to divide something by zero, and dividing a negative number doesn't make any sense - you can't pay negative income on rent!


Let's create a boolean that only contains incomes greater than zero, and then use that boolean to filter our `pumf_summary` data frame:

In [ ]:
# Create a boolean for positive incomes
positive_income = pumf_summary["Annual Income"] > 0

# Filter subset_pumf_data using our `actual_income` boolean
pumf_summary = pumf_summary[positive_income]

# Describe Annual Income column
pumf_summary[["Annual Income"]].describe()

Now, the minimum household income is \\$1!

## Step 7: Convert Annual Income to Monthly Income

Currently, we have annual income and monthly housing costs. To compare the two, we need to change annual income so that it is also monthly! Let's create a new column called `Monthly Household Income`, where the annual income is divided by 12.



To create a new column, we can put the name of the new column in `[]` brackets, and then provide the new values you want that column to have.

The calculation of the values is done **element-wise**. This means all values in the original column (`Annual Income`) are divided by 12. You do not need to use a loop to iterate each of the rows!

In [ ]:
pumf_summary["Monthly Household Income"] = pumf_summary["Annual Income"] / 12

pumf_summary.head()

## Step 8: Calculate Shelter-Cost-to-Income Ratio

Finally, we are ready to calculate how much of each household's annual income is spent on housing! Let's create an additional column that divides `Monthly Housing Costs` by `Monthly Household Income`:

$${\text{Shelter-Cost-to-Income Ratio}} = 100\times \frac{\text{Monthly Shelter Cost}}{\text{Monthly Household Income}} $$


In [ ]:
pumf_summary["Shelter-Cost-To-Income Ratio"] = 100*(pumf_summary["Monthly Housing Costs"]/pumf_summary["Monthly Household Income"])

pumf_summary.head()

Household 1 spends 20% of its annual income on housing, Household 2 spends 4.9% of its annual income on housing, etc.

## Step 9: "Topcoding" high values

If we use `.describe()` on our new column, there is something strange - some households appear to be paying way more than 100% of their income on housing! That shouldn't be possible.

In [ ]:
pumf_summary[["Shelter-Cost-To-Income Ratio"]].describe()

Let's investigate the highest value to see what's going on:

In [ ]:
pumf_summary[pumf_summary["Shelter-Cost-To-Income Ratio"] == 5780400]

It looks like this is happening because there are some households with an Annual Income of only $1! This is resulting in extremely high values that don't make any sense. Realistically, households can only spend 100% of their income on rent. 

To fix this, we will need to "topcode" this variable so that any values greater than 100% are automatically reset to be 100%.

In [ ]:
pumf_summary.loc[pumf_summary["Shelter-Cost-To-Income Ratio"] > 100, "Shelter-Cost-To-Income Ratio"] = 100

If we `.describe()` this column again, we can see that the maximum value is now 100%!

In [ ]:
pumf_summary["Shelter-Cost-To-Income Ratio"].describe()

## Step 10: Visualize our data

Let's create a boxplot to visualize our new `Shelter-Cost-To-Income Ratio` variable. This provides us with information about our overall distribution, as well as where the median, 1st quartile, and 3rd quartile are located:

In [ ]:
pumf_summary.plot.box(column = "Shelter-Cost-To-Income Ratio")

We can also break this down into different metro areas!

In [ ]:
pumf_summary.plot.box(column = "Shelter-Cost-To-Income Ratio", by = "Metro Area")

It isn't very intuitive to use the internal numeric codes for these metro areas. Let's use `.loc()` to change their names. For example, we can change the value of `Metro Area` to "Montreal" anytime that column is equal to 462:

In [ ]:
pumf_summary.loc[pumf_summary["Metro Area"] == 462, "Metro Area"] = "Montreal"

pumf_summary.head()

Let's change all the rest of the `Metro Area` labels

In [ ]:
pumf_summary.loc[pumf_summary["Metro Area"] == 535, "Metro Area"] = "Toronto"

pumf_summary.loc[pumf_summary["Metro Area"] == 825, "Metro Area"] = "Calgary"

pumf_summary.loc[pumf_summary["Metro Area"] == 835, "Metro Area"] = "Edmonton"

pumf_summary.loc[pumf_summary["Metro Area"] == 933, "Metro Area"] = "Vancouver"

pumf_summary.loc[pumf_summary["Metro Area"] == 999, "Metro Area"] = "Elsewhere"

Finally, let's redo our plot!

In [ ]:
pumf_summary.plot.box(column = "Shelter-Cost-To-Income Ratio", by = "Metro Area")